In [1]:
from qat.qlmaas import QLMaaSConnection
conn = QLMaaSConnection(hostname="qlm35e.neasqc.eu", check_host=False)

In [2]:
import parameters
from SQA_hyper_tuning_BO import BayesianOptimizer, SQA_solver, Brute_Force_Solver
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

In [3]:
n_spins, n_shots, delta_t = parameters.n_spins, parameters.n_shots, parameters.delta_t
n_init, n_iters = parameters.n_init, parameters.n_iters
min_vals, max_vals = parameters.min_vals, parameters.max_vals
bounds = [(min_vals[i], max_vals[i]) for i in range(len(min_vals))]

In [ ]:
sqa = SQA_solver(n_spins=n_spins, n_shots=n_shots, delta_t=delta_t)

num_repeats = 5  # Number of times to repeat the optimization with different seeds
mode = 'graph' # "graph" or "random"

if mode =='random':
    # Generate a random Ising problem (same for all runs)
    J, h = sqa.generate_random_problems(n_examples=1, seed=1)
elif mode == 'graph':
    file_name = "40nodes_p0.3_J-1.0to1.0_h0_Erdös-Rényi.pkl"
    G = sqa.load_graph_from_file(filepath=f"./{file_name}")
    J, h = sqa.ising_params_from_graph(G)

seed_list = [i for i in range(1, num_repeats + 1)]

# Repeat the parameters for each seed
n_spins_list = [n_spins] * num_repeats
n_init_list = [n_init] * num_repeats
n_iters_list = [n_iters] * num_repeats
delta_t_list = [delta_t] * num_repeats
n_shots_list = [n_shots] * num_repeats

beta_list = []
annealing_time_list = []
gamma_0_list = []
n_trotters_list = []
energy_list = []
exact_gs_energy_list = []
bf_time_list = []
time_list = []
for seed in seed_list:
    print(f"Running Bayesian optimization with seed {seed}...")
    start = time.time()
    bo = BayesianOptimizer(sqa=sqa, bounds=bounds, n_init=n_init, seed=seed)
    X_opt, Y_opt, history = bo.optimize(n_iters=n_iters)
    best_idx = np.argmin(Y_opt)
    end = time.time()
    beta_list.append(X_opt[best_idx][0])
    annealing_time_list.append(X_opt[best_idx][1])
    gamma_0_list.append(X_opt[best_idx][2])    
    n_trotters_list.append(int(X_opt[best_idx][3]))
    energy_list.append(Y_opt[best_idx][0])
    time_list.append(end - start)
    print(f"Best parameters for seed {seed}: beta={X_opt[best_idx][0]:.4f}, annealing_time={X_opt[best_idx][1]:.4f}, gamma_0={X_opt[best_idx][2]:.4f}, n_trotters={int(X_opt[best_idx][3])}")
    print(f"SQA ground state energy: {Y_opt[best_idx][0]:.4f}")


start_bf = time.time()
bf_solver = Brute_Force_Solver(J, h)
exact_gs_energy, config = bf_solver.brute_force_solve()
end_bf = time.time()
bf_time_list = [end_bf - start_bf]*num_repeats
exact_gs_energy_list = [exact_gs_energy]*num_repeats
print(f"Actual ground state energy: {exact_gs_energy:.4f}")
print(f"Brute force solver execution time: {bf_time_list[0]:.4f} seconds")

df = pd.DataFrame(list(zip(n_spins_list, n_shots_list, delta_t_list, n_init_list, n_iters_list, seed_list, time_list, energy_list, beta_list, annealing_time_list, gamma_0_list, n_trotters_list, exact_gs_energy_list, bf_time_list)), 
                  columns=["n_spins", "n_shots", "delta_t", "n_init", "n_iters", "seed", "execution_time", "min_energy", "opt_beta", "opt_annealing_time", "opt_gamma_0", "opt_n_trotters", "exact_gs_energy", "bf_time"])
path = f'C:/Users/a943763\OneDrive - ATOS/Documentos/Python/SQA/SQA_Tunning/Codes/Laboratory/SQA_tunning_temp_gamma_0_v1/ML_hyp_tuning/tables/BO_SQA_results_{n_spins}_fixed_problem_bf_{n_init}_{n_iters}_new_max_t.csv'
df.to_csv(path)

Running Bayesian optimization with seed 1...
Submitted a new batch: SJob44089
Submitted a new batch: SJob44090
Submitted a new batch: SJob44091


KeyboardInterrupt: 